1. TF group에서 어떤 값응ㄹ 추적할 것인지 결정한다.
2. mergy하여 summary한다.
3. 작성할 파일을 만들고 그래프를 쓴다.
4. summary를 실행시킨다.
5. 텐서보드를 실행시킨다.

## TensorBoard : logging/debugging tool
- Scalar
- Histogram : multi dimensional tensors
- Scope
- multiple run
tensorboard -logdir=./logs/xor_logs
http://127.0.0.1:6006
ssh -L local_port:127.0.0.1:remote_port username@server.com

In [1]:
# Lab 9 XOR
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    # 1-1. From TF graph, decide which tensors you want to log
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    # 1-2. From TF graph, decide which tensors you want to log
    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    # 1-3 From TF graph, decide which tensors you want to log
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
# 1-4 From TF graph, decide which tensors you want to log
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # 2. merge all summaries
    merged_summary = tf.summary.merge_all()
    # 3. create writer and add graph
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # 4. run summary merge and add_summary
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    # 5. Launch TensorBoard
    # tensorboard --logdir=./logs

0 0.715638 [array([[ 0.79269588,  0.68861037],
       [-1.20728338, -0.29517072]], dtype=float32), array([[ 1.7177    ],
       [ 0.35572481]], dtype=float32)]
100 0.690733 [array([[ 0.32268226, -0.23303065],
       [-0.7743414 ,  0.80529422]], dtype=float32), array([[ 1.46081114],
       [ 0.78415203]], dtype=float32)]
200 0.495032 [array([[ 1.67216849, -2.3216548 ],
       [-2.21686292,  2.656147  ]], dtype=float32), array([[ 2.82439065],
       [ 2.60258102]], dtype=float32)]
300 0.223746 [array([[ 3.18776274, -3.74428797],
       [-3.69724274,  4.12956715]], dtype=float32), array([[ 4.70040751],
       [ 4.31880617]], dtype=float32)]
400 0.122295 [array([[ 3.98867226, -4.52083349],
       [-4.4841404 ,  4.90343523]], dtype=float32), array([[ 5.8487258],
       [ 5.4189949]], dtype=float32)]
500 0.0788432 [array([[ 4.48614597, -5.01518679],
       [-4.97325087,  5.39076996]], dtype=float32), array([[ 6.64474821],
       [ 6.20202589]], dtype=float32)]
600 0.0559812 [array([[ 4.83938

5000 0.000789405 [array([[ 7.84969902, -8.45132351],
       [-8.30820179,  8.76673508]], dtype=float32), array([[ 15.14112663],
       [ 14.76316166]], dtype=float32)]
5100 0.000746024 [array([[ 7.87877464, -8.48109436],
       [-8.33723068,  8.79612732]], dtype=float32), array([[ 15.24907398],
       [ 14.87186337]], dtype=float32)]
5200 0.000705329 [array([[ 7.90747595, -8.51048183],
       [-8.3658886 ,  8.82514381]], dtype=float32), array([[ 15.35638905],
       [ 14.97991753]], dtype=float32)]
5300 0.000667051 [array([[ 7.93581915, -8.53950119],
       [-8.39418983,  8.85379791]], dtype=float32), array([[ 15.46310902],
       [ 15.0873661 ]], dtype=float32)]
5400 0.000631071 [array([[ 7.96382046, -8.56816387],
       [-8.42215729,  8.88210297]], dtype=float32), array([[ 15.5692749 ],
       [ 15.19424152]], dtype=float32)]
5500 0.000597195 [array([[ 7.99149323, -8.59649086],
       [-8.44979191,  8.91008091]], dtype=float32), array([[ 15.67490768],
       [ 15.30058098]], dtype=fl

10000 5.83398e-05 [array([[ 9.02439308, -9.65184498],
       [-9.48265171,  9.95404053]], dtype=float32), array([[ 20.17106056],
       [ 19.82086754]], dtype=float32)]

Hypothesis:  [[  6.13103184e-05]
 [  9.99936938e-01]
 [  9.99950767e-01]
 [  5.97514772e-05]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0
